In [49]:
!pip install mediapipe

In [50]:
!pip install pyautogui

# Import Libraries and Define Global Variables

In [43]:
import cv2
import mediapipe as mp
import pyautogui
import tkinter as tk
from PIL import Image, ImageTk

# Initialize variables
x1 = y1 = x2 = y2 = 0
dist = 0
running = False  # Flag to indicate if hand gesture detection is running


# Define Functions

In [44]:
# Function to start hand gesture detection
def start():
    global running
    running = True
    update()

# Function to stop hand gesture detection
def stop():
    global running
    running = False

# Function to update GUI with hand gesture detection
def update():
    global x1, y1, x2, y2, dist, running

    if running:
        _, image = webcam.read()
        image = cv2.flip(image, 1)
        frame_height, frame_width, _ = image.shape

        # Convert image to RGB for processing with MediaPipe
        rgb_image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        output = my_hands.process(rgb_image)
        hands = output.multi_hand_landmarks

        if hands:
            for hand in hands:
                drawing_utils.draw_landmarks(image, hand)
                landmarks = hand.landmark
                for id, landmark in enumerate(landmarks):
                    x = int(landmark.x * frame_width)
                    y = int(landmark.y * frame_height)
                    if id == 8:
                        cv2.circle(img=image, center=(x, y), radius=8, color=(0, 255, 255), thickness=3)
                        x1 = x
                        y1 = y
                    if id == 4:
                        cv2.circle(img=image, center=(x, y), radius=8, color=(0, 0, 255), thickness=3)
                        x2 = x
                        y2 = y

                dist = ((x2 - x1) ** 2 + (y2 - y1) ** 2) ** (0.5) // 4
                cv2.line(image, (x1, y1), (x2, y2), (0, 255, 0), 5)

            if dist > 50:
                pyautogui.press("volumeup")
            else:
                pyautogui.press("volumedown")

        # Display the image on the canvas
        img = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
        img = ImageTk.PhotoImage(image=img)
        canvas.img = img
        canvas.create_image(0, 0, anchor=tk.NW, image=img)

    root.after(10, update)  # Update the GUI every 10 milliseconds


# Initialize GUI and Run Application

In [54]:
# Tkinter GUI setup
root = tk.Tk()
root.title("Hand Gesture Control System")
root.configure(bg='#f0f0f0')

# Create a frame for the webcam feed
frame = tk.Frame(root, bg='#f0f0f0')
frame.pack(padx=10, pady=10)

# Create a label for the webcam feed
label = tk.Label(frame, text="Webcam Feed", font=("Helvetica", 12), bg='#f0f0f0')
label.pack()

# Create a canvas for displaying the webcam feed
canvas = tk.Canvas(frame, width=800, height=600, bg='white')
canvas.pack()

# Create start and stop buttons
start_button = tk.Button(root, text="Start", font=("Helvetica", 12), bg='#4CAF50', fg='white', command=start)
start_button.pack(side=tk.LEFT, padx=10, pady=10)

stop_button = tk.Button(root, text="Stop", font=("Helvetica", 12), bg='#F44336', fg='white', command=stop)
stop_button.pack(side=tk.LEFT, padx=10, pady=10)

# Initialize webcam
webcam = cv2.VideoCapture(0)
my_hands = mp.solutions.hands.Hands()
drawing_utils = mp.solutions.drawing_utils

root.mainloop()  # Start the Tkinter event loop
